# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fort st. john,56.2499,-120.8529,15.95,63,100,1.62,CA,1722658037
1,1,santo antonio do ica,-3.1022,-67.9397,23.63,85,49,1.14,BR,1722658370
2,2,ribeira grande,38.5167,-28.7000,22.18,78,10,1.83,PT,1722658371
3,3,edinburgh of the seven seas,-37.0676,-12.3116,9.52,50,69,10.66,SH,1722658373
4,4,umm kaddadah,13.6017,26.6876,27.52,61,76,3.49,SD,1722658374


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5
        
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_city = city_data_df.query("`Max Temp` <= 40 and Humidity >= 30 and Humidity <= 60")

# Drop any rows with null values
ideal_city.isnull().sum()

# Display sample data
ideal_city.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137 entries, 3 to 584
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     137 non-null    int64  
 1   City        137 non-null    object 
 2   Lat         137 non-null    float64
 3   Lng         137 non-null    float64
 4   Max Temp    137 non-null    float64
 5   Humidity    137 non-null    int64  
 6   Cloudiness  137 non-null    int64  
 7   Wind Speed  137 non-null    float64
 8   Country     133 non-null    object 
 9   Date        137 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 11.8+ KB


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city.loc[:,["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,50,
23,oranjemund,NaN,-28.5500,16.4333,43,
25,nogliki,RU,51.8333,143.1667,58,
27,blackmans bay,AU,-43.0167,147.3167,57,
31,howard springs,AU,-12.4970,131.0470,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories" : "accommodation.hotel",
    "limit" : 20,
    "apiKey" : geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]    
    lon = row["Lng"]    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
oranjemund - nearest hotel: Op My Stoep
nogliki - nearest hotel: Ноглики
blackmans bay - nearest hotel: Villa Howden
howard springs - nearest hotel: Rydges Palmerston
moraleja - nearest hotel: La Encomienda
whitehorse - nearest hotel: Town & Mountain Hotel
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
ifrane - nearest hotel: la perle d'azrou
ust-nera - nearest hotel: гостиница "Солнечная"
ola - nearest hotel: No hotel found
tongchuanshi - nearest hotel: No hotel found
waingapu - nearest hotel: HOTEL SANDLEWOOD
veydelevka - nearest hotel: Центральный
bargaal - nearest hotel: Makka Hotel Bargaal
yangliuqing - nearest hotel: IU Inn
dourados - nearest hotel: Hotel Figueiras
avarua - nearest hotel: Paradise Inn
ulladulla - nearest hotel: Hotel Marlin
letlhakeng - nearest hotel: No hotel found
sidi aissa - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
thompson - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,50,No hotel found
23,oranjemund,NaN,-28.5500,16.4333,43,Op My Stoep
25,nogliki,RU,51.8333,143.1667,58,Ноглики
27,blackmans bay,AU,-43.0167,147.3167,57,Villa Howden
31,howard springs,AU,-12.4970,131.0470,45,Rydges Palmerston
...,...,...,...,...,...,...
555,phan rang-thap cham,VN,11.5667,108.9833,49,Khách Sạn Ninh Thuận
561,mwene-ditu,CD,-7.0000,23.4500,32,Hôtel KAMAS
565,zheleznogorsk-ilimskiy,RU,56.5768,104.1217,40,Магнетит
569,mednogorsk,RU,51.4128,57.5950,52,Центральная


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    scale = 1.5    ,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
    
    
    
    
    
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)